In [4]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torchvision
from torch.nn import functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook, tqdm

##### 定义初始参数及激活函数

In [19]:
inputs = np.array([3, 5])
weights = {'h11': np.array([2, 4]),
           'h12': np.array([4, -5]),
           'h21': np.array([-1, 1]),
           'h22': np.array([2, 2]),
           'out': np.array([-3, 7])}

def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

##### 逐层计算神经网络输出
1. 首先是第一个隐藏层，你需要将输入层的数据与隐藏层的权重相乘、求和、并输入到激活函数中。

In [35]:
# hidden_11_value = tanh((inputs * weights['h11']).sum())
# hidden_12_value = tanh((inputs * weights['h12']).sum())
# hidden_1_output = np.array([hidden_11_value, hidden_12_value])

hidden_11_value = tanh(inputs.dot(weights['h11']).sum())
hidden_12_value = tanh(inputs.dot(weights['h12']).sum())
hidden_1_output = np.array([hidden_11_value, hidden_12_value])

2. 接下来是第二个隐藏层，这一层的操作与上一层完全相同。

In [36]:
hidden_21_value = tanh(hidden_1_output.dot(weights['h21']).sum())
hidden_22_value = tanh(hidden_1_output.dot(weights['h22']).sum())
hidden_2_output = np.array([hidden_21_value, hidden_22_value])

3. 最后是输出层，此时只有一个节点需要运算，且无需添加激活函数。

In [37]:
output = (hidden_2_output * weights['out']).sum()

In [38]:
output

2.8920827403683393

### CIFAR-10 图像分类过程
##### 定义对图像的各种变换操作，包括把array转换为tensor，对图像做正则化

In [ ]:
# transforms.Compose主要是用于常见的一些图形变换，例如裁剪、旋转
# 遍历list数组，对img依次执行每个transforms操作
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.48216, 0.44653),
                                                     (0.24703, 0.24349, 0.26159))])
train_set = torchvision.datasets.CIFAR10(
    root='./data/cifar10-data/', train=True,
    download=True, transform=transform
)
test_set = torchvision.datasets.CIFAR10(
    root='./data/cifar10-data/',
    download=True, transform=transform
)

# 用来把训练数据分成多个小组，此函数每次抛出一组数据。
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=16,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=16,
                                          shuffle=False)
train_set.classes

In [ ]:
# 把图片进行可视化展示
def imshow(inp, title=None):
    fig = plt.figure(figsize=(30, 30))
    # 转换成图片的维度
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    # 对图片进行标准化
    inp = std * inp + mean
    # 整个图片数组的值限制在a_min 与 a_max之间
    inp = np.clip(inp, 0, 1)
    # 对图片进行可视化展示
    plt.imshow(inp, )

# 获取一个batch的数据
inputs, classes = next(iter(train_loader))
# 以网格展示，作用是将若干幅图拼成一幅图像
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[train_set.classes[x] for x in classes])

##### 搭建简单的神经网络

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.relu(self.fc3(x))

net = Net()
# 定义损失函数-交叉熵
criterion = nn.CrossEntropyLoss()
# 定义优化器， 将神经网络的参数都传入优化器，并定义学习率
optimizer = optim.Adam(net.parameters(), lr=3e-4)

##### 神经网络的训练步骤
1. 大for循环-epochs，用于管理一套数据循环训练多少遍<br>
2. 小for循环-step，用于以batch_size为单位，从dataloader中调取数据
3. 清空优化器的梯度
4. 读入data, label，并进行形状转换
5. 运行模型前向传播过程
6. 基于模型输出生成最终结果
7. 计算损失
8. 基于损失计算梯度
9. 基于梯度更新参数

In [ ]:
num_epochs = 10
since = time.time()
net.train()
# 1. 用于管理一套数据循环训练多少遍
for epoch in range(num_epochs):
    print(f'Epoch: {epoch} / {num_epochs}')
    running_loss = 0.0
    running_corrects = 0
    # 2. 从trainloader里循环取出每一批次数据，
    for data in tqdm_notebook(train_loader):
        # 3. 清空优化器梯度
        optimizer.zero_grad()
        # 4. 读入data, label，并进行形状转换
        inputs, labels = data
        inputs = inputs.view(-1, 32 * 32 * 3)
        # 5. 运行模型前向传播过程
        outputs = net(inputs)
        # 6. 基于模型输出生成最终结果
        _, preds = torch.max(outputs, 1)
        # 7. 计算损失
        loss = criterion(outputs, labels)
        # 8. 基于损失计算梯度
        loss.backward()
        # 9. 基于梯度更新参数
        optimizer.step()

        # 一个批次数据的损失函数的计算
        running_loss += loss.items() * inputs.size(0)
        # 一个批次数据准确率的计算
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / train_loader.dataset.data.shape[0]
    epoch_acc = running_corrects.double() / train_loader.dataset.data.shape[0]
    print('Train loss: {:.4f}, Acc: {:.4f'.format(epoch_loss, epoch_acc))
    print('-' * 50)
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'
      .format(time_elapsed // 60, time_elapsed % 60))


##### 模型测评
模型测评过程中的数据导入、前向传播过程与训练过程基本相同，可以参照训练过程来写


In [ ]:
correct = total = 0
net.eval()
for data in tqdm_notebook(test_loader):
    # 1. 数据导入
    inputs, labels = data
    inputs = inputs.view(-1, 32 * 32 * 3)
    # 前向传播
    outputs = net(inputs)

    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))